-----
# 2021 AI/ML 개발 신입(석/박사) 공채 Vision Part 과제


# 과제설명

# 개요
* Classfication 모델 구현 과제
    * 해당 과제는 크게 두 가지 Task로 나누어 진행됩니다.
    * 각 Task에 명시된 데이터 셋을 사용하여 과제를 수행하고 양식에 따라 결과를 제출합니다.
        * `vision.zip` 구성
            * `images/` 이미지 파일 폴더
            * `task#_(train,test).csv` 데이터셋
            * `report.ipynb` jupyter notebook (과제 설명 및 레포트 양식 포함)

## Task1

### Lightweight classfication 모델 구현
주어신 데이터셋을 분류하는 모델을 구현하세요.

* Dataset
    * `task1_train.csv`: 학습 데이터
    * `task1_test.csv`: 평가 데이터
    * Format: (Image file name),(Predicted Label)
* 평가 항목: 
    * Accuracy
    * 연산량 및 파라미터 수

단, 구현 모델은 다음 제약조건을 준수하여야 합니다.
* 연산량: **30 MFLOPs 이하** 
    * convolution, linear 등 주요 layer만 계산 가능
    * 계산 방법 설명 필요
* 파라미터 수: **2 MB 이하** 
    * numpy.float32 기준이나 다른 data type 적용 가능
* Loss function: **Cross Entropy 사용**
* **ML framework 및 data analysis용 library 사용 금지, numpy 사용 가능**
    * TensorFlow, PyTorch, JAX, SciPy, scikit-learn, pandas 등 사용 금지
    * Utility library 사용 가능


## Task2

### Robust classfication 모델 구현

다음 reference를 참고하여 새로운 데이터셋을 분류하는 모델을 구현하세요.
* Symmetric Cross Entropy Loss (https://arxiv.org/pdf/1908.06112.pdf)
* Improved Mean Absolute Error (https://arxiv.org/pdf/1903.12141v6.pdf)
* Barrier Hinge Loss (https://arxiv.org/pdf/1901.09314.pdf)


* Dataset
    * `task2_train.csv`: 학습 데이터
    * `task2_test.csv`: 평가 데이터
    * Format: (Image file name),(Predicted Label)
* 평가 항목: 
    * Accuracy
    * 연산량 및 파라미터 수
  
단, 구현 모델은 다음 제약조건을 준수하여야 합니다.
* 연산량: **30 MFLOPs 이하** 
    * convolution, linear 등 주요 layer만 포함하여 계산 가능
    * 계산 방법 설명 필요
* 파라미터 수: **2 MB 이하** 
    * numpy.float32 기준이나 다른 data type 적용 가능
* Loss function: **제약 없음**
* **ML framework 및 data analysis용 library 사용 금지, numpy 사용 가능**
    * TensorFlow, PyTorch, JAX, SciPy, scikit-learn, pandas 등 사용 금지
    * Utility library 사용 가능


# 제출항목
* 구현 코드
    * 코드 내 주석 및 모듈화 필수
* 평가 데이터에 대한 분류 결과 파일
    * `$PROJECT_ROOT/task1_result.csv`, `$PROJECT_ROOT/task2_result.csv`
    * Format: (Image file name),(Predicted Label)
        * `test.csv` 참조
* 과제에 대한 레포트 jupyter notebook 
    * `$PROJECT_ROOT/report.ipynb`
    * 주어진 양식을 활용하여 작성
    * 필요 프레임워크 등의 코드는 따로 작성 후 학습 결과만 첨부하거나, 노트북 내에서 학습 진행 가능
    * 레포트 항목을 시각화하기 위한 라이브러리 제한은 없음
    * 설명을 위해 셀이 추가적으로 필요한 경우 자유롭게 작성


----
# Task1 Report


## 결과

* 작성 코드에 대한 설명과 실행방법을 정리해주세요.

In [10]:
'''
ML framework를 이용하지 않고 ResNet 기반의 모델을 구현하여 주어진 데이터셋에 대해 학습, 평가, 및 추론하였습니다.

각각의 코드에 대하여 설명드리겠습니다.


- dataloader.py

학습을 위해 주어진 csv 파일을 읽고, 이미지 폴더로부터 batch 단위로 데이터를 불러오는 코드입니다.

데이터를 전부 학습하면 reset()을 통해 데이터셋을 셔플하고 처음부터 다시 불러옵니다.


- evaluate.py

평가를 위해 테스트용 csv 파일을 읽고, 이미지로부터 결과를 추출하여 정답과 비교하는 코드입니다.

결과에 대한 전체 정확도 및 카테고리 별 정확도를 반환합니다.

평가를 통해 정확도가 가장 높은 모델만 저장하게 됩니다.


- inference.py

학습된 모델을 불러와서 테스트 데이터셋에 대한 결과를 추론하는 코드입니다.

이미지 폴더, 테스트 데이터셋에 대한 정보가 담긴 csv 파일, label mapping을 위한 카테고리 파일, 모델 파일, 결과 csv 파일을 저장할 위치, batch 크기를 인자로 받습니다.

image_classification 폴더에서 아래와 같은 형식으로 코드를 실행하면, 추론 결과를 save_pth에 저장하며 정확도를 출력해줍니다.

python inference.py --image_dir='/vision/images/' --csv_pth='/vision/task1_test.csv' --category_pth='category_task1.txt' --model_pth='savedmodel/task1' --save_pth='../task1_result.csv' --batch_size=20

=> Total accuracy : 80.604
=> Class 'Watches' accuracy :  96.850
=> Class 'Shoes' accuracy :  98.400
...

- layer.py

모델을 구성하는 여러 layer들에 대한 코드입니다.

Fully-connected, 2D convolution, max pooling, gloval average pooling, batch normalization 및 ReLU 함수로 구성되어 있습니다.

각 layer에 대한 forward, backward 연산을 통해 output을 전달하고 kernel 및 gradient를 업데이트 해주었습니다.

Fully-connected 와 2D convolution layer의 파라미터는 He uniform initialization 방법을 통해 초기화해주었습니다.

Fully-connected 와 2D convolution layer는 파라미터를 저장 및 불러올 수 있게 save 및 load 함수를 포함하고 있습니다.


- make_category.py

학습 데이터를 먼저 읽고 카테고리 맵을 생성하는 코드입니다. 생성한 카테고리 맵을 이용해서 label을 그에 맞는 정수값으로 매핑해줍니다.

생성한 카테고리 맵은 저장되어 학습 또는 추론에 이용됩니다.


- model.py

학습에 사용한 MyResNet 모델에 대한 코드입니다.

모델은 ResNet을 기반으로 구현되었으며, 연산량 및 모델 크기에 대한 제한 조건에 맞게 구성되었습니다.

모델을 ResBlock들로 구성되었는데, skip-connection을 통해 gradient를 잘 전파할 수 있도록 설계되었습니다.


- train.py

Dataloader를 통해 데이터를 불러오고, Trainer를 통해 모델 학습을 진행하는 코드입니다.

학습이 진행됨에 따라 정확도 및 Loss의 크기가 기록되며, 학습된 모델은 정해진 경로에 저장됩니다.

이미지 폴더, 학습 및 테스트 데이터셋에 대한 정보가 담긴 csv 파일, label mapping을 위한 카테고리 파일, 모델을 저장할 위치, batch 크기, epoch 수를 인자로 받습니다.

image_classification 폴더에서 아래와 같은 형식으로 코드를 실행하면, 학습이 시작됩니다.

python train.py --image_dir='/vision/images/' --train_csv_pth='/vision/task1_train.csv' --test_csv_pth='/vision/task1_test.csv' --category_pth='category_task1.txt' --model_save_pth='savedmodel/task1' --batch_size=20 --epochs=30


- trainer.py

손실 함수(Cross entropy loss)를 정의하고, backpropagation을 통해 학습을 시작하는 코드입니다.


- utils.py

학습 및 추론에 필요한 여러가지 함수들을 포함하는 코드입니다.

이미지 전처리, 이미지 패딩, csv파일 읽어서 리스트 형태로 변환 등을 포함합니다.


'''

"\nML framework를 이용하지 않고 ResNet 기반의 모델을 구현하여 주어진 데이터셋에 대해 학습, 평가, 및 추론하였습니다.\n\n각각의 코드에 대하여 설명드리겠습니다.\n\n\n- dataloader.py\n\n학습을 위해 주어진 csv 파일을 읽고, 이미지 폴더로부터 batch 단위로 데이터를 불러오는 코드입니다.\n\n데이터를 전부 학습하면 reset()을 통해 데이터셋을 셔플하고 처음부터 다시 불러옵니다.\n\n\n- evaluate.py\n\n평가를 위해 테스트용 csv 파일을 읽고, 이미지로부터 결과를 추출하여 정답과 비교하는 코드입니다.\n\n결과에 대한 전체 정확도 및 카테고리 별 정확도를 반환합니다.\n\n평가를 통해 정확도가 가장 높은 모델만 저장하게 됩니다.\n\n\n- inference.py\n\n학습된 모델을 불러와서 테스트 데이터셋에 대한 결과를 추론하는 코드입니다.\n\n이미지 폴더, 테스트 데이터셋에 대한 정보가 담긴 csv 파일, label mapping을 위한 카테고리 파일, 모델 파일, 결과 csv 파일을 저장할 위치, batch 크기를 인자로 받습니다.\n\n아래와 같은 형식으로 코드를 실행하면, 추론 결과를 save_pth에 저장하며 정확도를 출력해줍니다.\n\npython inference.py --image_dir='/vision/images/' --csv_pth='/vision/task1_test.csv' --category_pth='category_task1.txt' --model_pth='savedmodel/task1' --save_pth='../task1_result.csv' --batch_size=20\n\n=> Total accuracy : 80.604\n=> Class 'Watches' accuracy :  96.850\n=> Class 'Shoes' accuracy :  98.400\n...\n\n- layer.py\n\n모델을 구성하는 여러 layer들에 대한 코드입니다.\n\nFully-c

* 작성 모델에 관해 설명해주세요.

In [7]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'''
- 모델 구조

ResNet 기반의 모델을 이용하였습니다.

MyResNet은 7x7 Conv2D와 Maxpooling Layer로 이루어진 HeadBlock과 5개의 ResBlock, 그리고 GlobalAveragePooling 및 FullyConnected Layer로 구성되어 있습니다.

ResBlock은 3x3 Conv2D - BN Layer - ReLU 가 두 번 연결되어있는 구조를 가집니다.

또한 1, 2, 4번째 ResBlock에는, Block의 input tensor을 마지막 BN Layer의 output tensor에 더해주는 skip connection이 있습니다.

이 때 input tensor의 차원을 BN Layer의 output tensor에 맞춰주기 위해, skip connection은 하나의 1x1 Conv2D와 BN을 거치게 됩니다.

5개의 ResBlock을 거치고 나면, GlobalAveragePooling Layer를 통해 feature의 크기를 1x1로 줄여주고 channel 수만큼의 크기를 가지는 벡터로 변환할 수 있게 됩니다.

이후 FullyConnected 및 Softmax Layer를 통해 각 카테고리에 대한 확률분포를 얻게 됩니다.


- 모델 크기 및 연산량

모델은 총 14개의 Conv2D Layer와 1개의 FullyConnected Layer로 구성되어 있습니다.

모델의 입력의 크기를 80x80으로 고정하면, 각 Conv2D Layer의 (커널 크기(정방형), 입력 채널수, 출력 채널수, 출력 크기)는 아래와 같습니다. (sc는 skip connection을 의미합니다.)

Conv2D의 경우 파라미터의 수는 (커널 크기 * 입력 채널수 * 출력 채널수)로 결정되며, 연산량은 (커널 크기 * 입력 채널수 * 출력 채널수 * 출력 크기)로 결정됩니다.

FullyConnected의 경우 파라미터 수는 (입력 채널수 * 출력 채널수)로 결정되며, 연산량은 ((2*입력 채널수-1) * 출력 채널수)로 결정됩니다.

Pooling의 경우 파라미터 수는 0이며, 연산량은 (커널 크기 * 출력 채널 수 * 출력 크기)로 결정됩니다.

# Layers
conv1(Headblock): (7, 7, 3, 32, 40, 40)  
conv2(ResBlock1_1): (3, 3, 32, 32, 20, 20)
conv3(ResBlock1_2): (3, 3, 32, 32, 20, 20)
conv4(ResBlock1_sc): (1, 1, 32, 32, 20, 20)
conv5(ResBlock2_1): (3, 3, 32, 32, 10, 10)
conv6(ResBlock2_2): (3, 3, 32, 32, 10, 10)
conv7(ResBlock2_sc): (1, 1, 32, 32, 10, 10)
conv8(ResBlock3_1): (3, 3, 32, 32, 10, 10)
conv9(ResBlock3_2): (3, 3, 32, 32, 10, 10)
conv10(ResBlock4_1): (3, 3, 32, 64, 5, 5)
conv11(ResBlock4_2): (3, 3, 32, 64, 5, 5)
conv12(ResBlock4_sc): (1, 1, 32, 64, 5, 5)
conv13(ResBlock5_1): (3, 3, 64, 64, 5, 5)
conv14(ResBlock5_2): (3, 3, 64, 64, 5, 5)
fully-connected: (64, 22(num_classes of task1))
maxpooling: (3, 3, 32, 20, 20)
global-averagepooling: (5, 5, 64, 1, 1)

Conv2D와 FullyConnect는 He uniform initialization을 통해 초기화하는데, 이 때 사용하는 np.random.uniform 함수의 기본 dtype이 float64입니다.

따라서 파라미터는 개당 8Byte를 가지며, 이를 기준으로 총 파라미터 크기를 계산하였습니다.
'''

dict_layers = {}
dict_layers['conv'] = [(7, 7, 3, 32, 40, 40) , (3, 3, 32, 32, 20, 20), (3, 3, 32, 32, 20, 20), (1, 1, 32, 32, 20, 20), (3, 3, 32, 32, 10, 10),
                      (3, 3, 32, 32, 10, 10), (1, 1, 32, 32, 10, 10), (3, 3, 32, 32, 10, 10), (3, 3, 32, 32, 10, 10), (3, 3, 32, 64, 5, 5), 
                      (3, 3, 32, 64, 5, 5), (1, 1, 32, 64, 5, 5), (3, 3, 64, 64, 5, 5), (3, 3, 64, 64, 5, 5)]
dict_layers['fc'] = [(64, 22)]
dict_layers['pooling'] = [(3, 3, 32, 20, 20), (5, 5, 64, 1, 1)]

total_flops, total_params = 0, 0
for key in dict_layers.keys():
    if key=='conv':
        for layer in dict_layers[key]:
            total_flops += layer[0]*layer[1]*layer[2]*layer[3]*layer[4]*layer[5]
            total_params += layer[0]*layer[1]*layer[2]*layer[3]
    elif key=='fc':
        for layer in dict_layers[key]:
            total_flops += (2*layer[0]-1)*layer[1]
            total_params += layer[0]*layer[1]
    else:
        for layer in dict_layers[key]:
            total_flops += layer[0]*layer[1]*layer[2]*layer[3]
            
print(f'총 연산량은 {total_flops:n} FLOPS 입니다.')
print(f'총 파라미터 크기는 {total_params * 8:n} B 입니다.')


총 연산량은 21,923,754 FLOPS 입니다.
총 파라미터 크기는 1,408,768 B 입니다.


* 학습 결과를 정리해주세요.

In [ ]:
'''
학습은 약 5epochs 정도 진행했습니다.

전체 및 카테고리별 정확도는 아래와 같습니다.

=> Total accuracy : 86.135
=> Class 'Watches' accuracy :  98.425
=> Class 'Shoes' accuracy :  97.600
=> Class 'Headwear' accuracy :  74.419
=> Class 'Jewellery' accuracy :  88.272
=> Class 'Bags' accuracy :  87.773
=> Class 'Dress' accuracy :  10.714
=> Class 'Fragrance' accuracy :  93.377
=> Class 'Saree' accuracy :  90.625
=> Class 'Bottomwear' accuracy :  93.052
=> Class 'Makeup' accuracy :  6.522
=> Class 'Nails' accuracy :  34.694
=> Class 'Wallets' accuracy :  88.489
=> Class 'Socks' accuracy :  82.692
=> Class 'Sandal' accuracy :  56.944
=> Class 'Topwear' accuracy :  96.600
=> Class 'Flip Flops' accuracy :  66.423
=> Class 'Loungewear and Nightwear' accuracy :  21.429
=> Class 'Ties' accuracy :  97.368
=> Class 'Belts' accuracy :  97.521
=> Class 'Lips' accuracy :  87.342
=> Class 'Eyewear' accuracy :  100.000
=> Class 'Innerwear' accuracy :  84.133


테스트 데이터셋에 대한 추론 결과('task1_result.csv')는 아래의 inference 함수를 통해 얻었습니다.

python inference.py --image_dir='/vision/images/' --csv_pth='/vision/task1_test.csv' --category_pth='category_task1.txt' --model_pth='savedmodel/task1' --save_pth='../task1_result.csv' --batch_size=20


'''

import pickle
import matplotlib.pyplot as plt


with open('image_classification/result/accuracy_task1.txt', 'rb') as file:
    accuracy = pickle.load(file)
with open('image_classification/result/loss_task1.txt', 'rb') as file:
    loss = pickle.load(file)
steps_loss, losses = [], []
for l in loss:
    steps_loss.append(l[0])
    losses.append(l[1])

steps_acc, total_acc = [], []
for a in accuracy:
    steps_acc.append(a[0])
    total_acc.append(a[1])

fig, ax1 = plt.subplots()
color = 'tab:blue'
ax1.set_xlabel('steps')
ax1.set_ylabel('loss')
ax1.plot(steps_loss, losses, color=color)
ax1.tick_params(axis='y')
ax2 = ax1.twinx()  
color = 'tab:orange'
ax2.set_ylabel('total accuracy (%)')  
ax2.plot(steps_acc, total_acc, color=color)
ax2.tick_params(axis='y')

fig.tight_layout() 
plt.show()


* 추가적인 논의사항이 있을 경우 서술해주세요.

In [ ]:
# ( ... )

----
# Task2 Report


## 결과

* 작성 코드에 대한 설명과 실행방법을 정리해주세요.

In [ ]:
'''
(Task1에서 이용한 코드를 기반으로 작성하였습니다.)

- trainer.py

import os
import csv
import json
import pickle
from tqdm import tqdm

from utils import log_clip
from dataloader import *
from model import *

def log_clip(tensor, A):
    tensor[tensor==0] = np.exp(A)
    return np.log(tensor)

def cross_entropy_loss(pred, label):
    batch_size = pred.shape[0]
    loss = np.sum(-label * np.log(pred)-(1-label) * np.log(1 - pred))/batch_size
    diff = (pred - label)/pred/(1 - pred)/batch_size
    return loss, diff

def reverse_cross_entropy_loss(pred, label, A):
    batch_size = pred.shape[0]
    loss = np.sum(-pred * log_clip(label, A)-(1-pred) * log_clip(1-label, A))/batch_size
    diff = (log_clip(1 - label, A)-log_clip(label, A))/batch_size
    return loss, diff


class trainer:
    def __init__(self, model, dataset, num_classes, init_lr):
        self.dataset = dataset
        self.net = model
        self.lr = init_lr
        self.cls_num = num_classes

    def set_lr(self, lr):
        self.lr = lr

    def iterate(self):
        images, labels = self.dataset.get_next_batch()

        out_tensor = self.net.forward(images)
        one_hot_labels = np.eye(self.cls_num)[(labels-1).reshape(-1)].reshape(out_tensor.shape)
            
        loss, out_diff_tensor = cross_entropy_loss(out_tensor, one_hot_labels)
        
        self.net.backward(out_diff_tensor, self.lr)
        
        return loss
    
    def iterate_symmetric(self, A, a, b):
        images, labels = self.dataset.get_next_batch()

        out_tensor = self.net.forward(images)
        one_hot_labels = np.eye(self.cls_num)[(labels-1).reshape(-1)].reshape(out_tensor.shape)
            
        ce_loss, ce_out_diff_tensor = cross_entropy_loss(out_tensor, one_hot_labels)
        rce_loss, rce_out_diff_tensor = reverse_cross_entropy_loss(out_tensor, one_hot_labels, A)
        
        total_loss = a*ce_loss + b*rce_loss
        out_diff_tensor = a*ce_out_diff_tensor + b*rce_out_diff_tensor
        
        self.net.backward(out_diff_tensor, self.lr)
        
        return total_loss

Symmetric Cross Entropy Loss를 정의하였습니다.(https://arxiv.org/pdf/1908.06112.pdf)

Noisy label에 tolerant한 Reverse Cross Entropy Loss를 정의함으로써 성능을 높이고, Cross Entropy를 더해줌으로써 수렴에 도움을 주었습니다.

One-hot encoding된 label tensor에 log를 바로 취하게 되면 음의 무한으로 발산하기 때문에, 논문에서 제안한 clipping을 구현해서 값이 발산하는 것을 방지하였습니다.

또한 Reverse Cross Entropy Loss의 미분값을 계산해서 gradient를 전파해주었습니다.

Trainer 부분에서는 iterate_symmetric(self, A, a, b) 함수를 통해 CE loss와 RCE loss를 적절히 더해주었습니다.

A, a, b 값은 논문에서 제안한대로 -6, 0.1, 1을 이용하였습니다.

Symmetric Learning은 train.py에 --symmetric_training=True 의 인자만 추가해주면 됩니다.

python train.py --image_dir='/vision/images/' --train_csv_pth='/vision/task2_train.csv' --test_csv_pth='/vision/task2_test.csv' --category_pth='category_task2.txt' --model_save_pth='savedmodel/task2' --batch_size=20 --epochs=30

'''

* 작성 모델에 관해 설명해주세요.

In [9]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'''
(Task1의 모델 설명과 같습니다.)

- 모델 구조

ResNet 기반의 모델을 이용하였습니다.

MyResNet은 7x7 Conv2D와 Maxpooling Layer로 이루어진 HeadBlock과 5개의 ResBlock, 그리고 GlobalAveragePooling 및 FullyConnected Layer로 구성되어 있습니다.

ResBlock은 3x3 Conv2D - BN Layer - ReLU 가 두 번 연결되어있는 구조를 가집니다.

또한 1, 2, 4번째 ResBlock에는, Block의 input tensor을 마지막 BN Layer의 output tensor에 더해주는 skip connection이 있습니다.

이 때 input tensor의 차원을 BN Layer의 output tensor에 맞춰주기 위해, skip connection은 하나의 1x1 Conv2D와 BN을 거치게 됩니다.

5개의 ResBlock을 거치고 나면, GlobalAveragePooling Layer를 통해 feature의 크기를 1x1로 줄여주고 channel 수만큼의 크기를 가지는 벡터로 변환할 수 있게 됩니다.

이후 FullyConnected 및 Softmax Layer를 통해 각 카테고리에 대한 확률분포를 얻게 됩니다.


- 모델 크기 및 연산량

모델은 총 14개의 Conv2D Layer와 1개의 FullyConnected Layer로 구성되어 있습니다.

모델의 입력의 크기를 80x80으로 고정하면, 각 Conv2D Layer의 (커널 크기(정방형), 입력 채널수, 출력 채널수, 출력 크기)는 아래와 같습니다. (sc는 skip connection을 의미합니다.)

Conv2D의 경우 파라미터의 수는 (커널 크기 * 입력 채널수 * 출력 채널수)로 결정되며, 연산량은 (커널 크기 * 입력 채널수 * 출력 채널수 * 출력 크기)로 결정됩니다.

FullyConnected의 경우 파라미터 수는 (입력 채널수 * 출력 채널수)로 결정되며, 연산량은 ((2*입력 채널수-1) * 출력 채널수)로 결정됩니다.

Pooling의 경우 파라미터 수는 0이며, 연산량은 (커널 크기 * 출력 채널 수 * 출력 크기)로 결정됩니다.

# Layers
conv1(Headblock): (7, 7, 3, 32, 40, 40)  
conv2(ResBlock1_1): (3, 3, 32, 32, 20, 20)
conv3(ResBlock1_2): (3, 3, 32, 32, 20, 20)
conv4(ResBlock1_sc): (1, 1, 32, 32, 20, 20)
conv5(ResBlock2_1): (3, 3, 32, 32, 10, 10)
conv6(ResBlock2_2): (3, 3, 32, 32, 10, 10)
conv7(ResBlock2_sc): (1, 1, 32, 32, 10, 10)
conv8(ResBlock3_1): (3, 3, 32, 32, 10, 10)
conv9(ResBlock3_2): (3, 3, 32, 32, 10, 10)
conv10(ResBlock4_1): (3, 3, 32, 64, 5, 5)
conv11(ResBlock4_2): (3, 3, 32, 64, 5, 5)
conv12(ResBlock4_sc): (1, 1, 32, 64, 5, 5)
conv13(ResBlock5_1): (3, 3, 64, 64, 5, 5)
conv14(ResBlock5_2): (3, 3, 64, 64, 5, 5)
fully-connected: (64, 22(num_classes of task1))
maxpooling: (3, 3, 32, 20, 20)
global-averagepooling: (5, 5, 64, 1, 1)

Conv2D와 FullyConnect는 He uniform initialization을 통해 초기화하는데, 이 때 사용하는 np.random.uniform 함수의 기본 dtype이 float64입니다.

따라서 파라미터는 개당 8Byte를 가지며, 이를 기준으로 총 파라미터 크기를 계산하였습니다.
'''

dict_layers = {}
dict_layers['conv'] = [(7, 7, 3, 32, 40, 40) , (3, 3, 32, 32, 20, 20), (3, 3, 32, 32, 20, 20), (1, 1, 32, 32, 20, 20), (3, 3, 32, 32, 10, 10),
                      (3, 3, 32, 32, 10, 10), (1, 1, 32, 32, 10, 10), (3, 3, 32, 32, 10, 10), (3, 3, 32, 32, 10, 10), (3, 3, 32, 64, 5, 5), 
                      (3, 3, 32, 64, 5, 5), (1, 1, 32, 64, 5, 5), (3, 3, 64, 64, 5, 5), (3, 3, 64, 64, 5, 5)]
dict_layers['fc'] = [(64, 22)]
dict_layers['pooling'] = [(3, 3, 32, 20, 20), (5, 5, 64, 1, 1)]

total_flops, total_params = 0, 0
for key in dict_layers.keys():
    if key=='conv':
        for layer in dict_layers[key]:
            total_flops += layer[0]*layer[1]*layer[2]*layer[3]*layer[4]*layer[5]
            total_params += layer[0]*layer[1]*layer[2]*layer[3]
    elif key=='fc':
        for layer in dict_layers[key]:
            total_flops += (2*layer[0]-1)*layer[1]
            total_params += layer[0]*layer[1]
    else:
        for layer in dict_layers[key]:
            total_flops += layer[0]*layer[1]*layer[2]*layer[3]
            
print(f'총 연산량은 {total_flops:n} FLOPS 입니다.')
print(f'총 파라미터 크기는 {total_params * 8:n} B 입니다.')


총 연산량은 21,923,754 FLOPS 입니다.
총 파라미터 크기는 1,408,768 B 입니다.


* 학습 결과를 정리해주세요.

In [ ]:
'''
학습은 약 5epochs 정도 진행했습니다.

전체 및 카테고리 별 정확도는 아래와 같습니다.

=> Total accuracy : 80.873
=> Class 'Watches' accuracy :  96.040
=> Class 'Shoes' accuracy :  98.148
=> Class 'Headwear' accuracy :  0.000
=> Class 'Jewellery' accuracy :  70.690
=> Class 'Cufflinks' accuracy :  0.000
=> Class 'Apparel Set' accuracy :  0.000
=> Class 'Stoles' accuracy :  0.000
=> Class 'Bags' accuracy :  95.868
=> Class 'Gloves' accuracy :  0.000
=> Class 'Dress' accuracy :  0.000
=> Class 'Fragrance' accuracy :  77.500
=> Class 'Saree' accuracy :  0.000
=> Class 'Mufflers' accuracy :  0.000
=> Class 'Bottomwear' accuracy :  75.556
=> Class 'Makeup' accuracy :  27.273
=> Class 'Nails' accuracy :  0.000
=> Class 'Wallets' accuracy :  57.500
=> Class 'Socks' accuracy :  38.462
=> Class 'Sandal' accuracy :  9.677
=> Class 'Hair' accuracy :  0.000
=> Class 'Topwear' accuracy :  97.893
=> Class 'Scarves' accuracy :  0.000
=> Class 'Flip Flops' accuracy :  51.429
=> Class 'Loungewear and Nightwear' accuracy :  16.667
=> Class 'Accessories' accuracy :  0.000
=> Class 'Ties' accuracy :  50.000
=> Class 'Belts' accuracy :  100.000
=> Class 'Eyes' accuracy :  0.000
=> Class 'Lips' accuracy :  0.000
=> Class 'Eyewear' accuracy :  97.619
=> Class 'Skin' accuracy :  0.000
=> Class 'Skin Care' accuracy :  0.000
=> Class 'Innerwear' accuracy :  51.852

테스트 데이터셋에 대한 추론 결과('task2_result.csv')는 아래의 inference 함수를 통해 얻었습니다.

python inference.py --image_dir='/vision/images/' --csv_pth='/vision/task2_test.csv' --category_pth='category_task2.txt' --model_pth='savedmodel/task2' --save_pth='../task2_result.csv' --batch_size=20


'''

import pickle
import matplotlib.pyplot as plt


with open('image_classification/result/accuracy_task2.txt', 'rb') as file:
    accuracy = pickle.load(file)
with open('image_classification/result/loss_task2.txt', 'rb') as file:
    loss = pickle.load(file)
steps_loss, losses = [], []
for l in loss:
    steps_loss.append(l[0])
    losses.append(l[1])

steps_acc, total_acc = [], []
for a in accuracy:
    steps_acc.append(a[0])
    total_acc.append(a[1])

fig, ax1 = plt.subplots()
color = 'tab:blue'
ax1.set_xlabel('steps')
ax1.set_ylabel('loss')
ax1.plot(steps_loss, losses, color=color)
ax1.tick_params(axis='y')
ax2 = ax1.twinx()  
color = 'tab:orange'
ax2.set_ylabel('total accuracy (%)')  
ax2.plot(steps_acc, total_acc, color=color)
ax2.tick_params(axis='y')

fig.tight_layout() 
plt.show()


* 추가적인 논의사항이 있을 경우 서술해주세요.

In [ ]:
# ( ... )